In [64]:
import boto3
import json
import turfpy
from turfpy.measurement import boolean_point_in_polygon
from geojson import Point, MultiPolygon, Feature
import pandas as pd
from io import StringIO

In [62]:
def read_json_from_s3(bucket_name, file_key):
    # Create an S3 client
    s3 = boto3.client('s3')

    try:
        # Retrieve the JSON file from S3
        response = s3.get_object(Bucket=bucket_name, Key=file_key)

        # Read the JSON data from the S3 response
        json_data = response['Body'].read().decode('utf-8')

        # Parse the JSON data into a DataFrame
        df = pd.read_json(StringIO(json_data))

        return df
    except Exception as e:
        print(f"Error reading JSON from S3: {e}")
        return None

In [52]:
def read_geojson_from_s3(bucket_name, file_key):
    s3 = boto3.client('s3')
    try:
        response = s3.get_object(Bucket=bucket_name, Key=file_key)
        geojson_data = response['Body'].read()
        return json.loads(geojson_data)
    except Exception as e:
        print(f"Error reading GeoJSON from S3: {e}")
        return None

In [90]:
bucket_name = 'mr-restuarant-bogota'
file_key = 'poblacion-upz-bogota.geojson'

In [94]:
geojson = read_geojson_from_s3(bucket_name, file_key)
print(geojson.keys())

dict_keys(['type', 'features'])


In [65]:
bucket_name = 'mr-restuarant-bogota'
file_key = 'restaurants_sample.json'

df = read_json_from_s3(bucket_name, file_key)
df

,name,rating,number_commnets_google,address,type,google_maps_url,latitud,longitud,hash_id
0,Restaurante Morena Parrilla,4.0,17.0,Cra. 12 # 15-25 Piso 2. Centro Comercial Gran...,Colombiana,https://www.google.com/maps/place/Restaurante+...,4.603997,-74.076967,027bcb6af9d1be568cf1ae518fa7c10f
1,RESTAURANTE GUSTA MAS,5.0,2.0,Cl. 16 #12-55,Colombiana,https://www.google.com/maps/place/RESTAURANTE+...,4.605092,-74.076780,c52a652d8a563dc0cd251ddf5814bbe2
2,Restaurante El Antojo de La 15,NaN,NaN,Cl. 15 #32,Restaurante,https://www.google.com/maps/place/Restaurante+...,4.603622,-74.076309,b6bc338fc1a8d79b22083f3d315dc6be
3,Restaurante Pescadero Del Centro,4.5,40.0,Cra. 9 #58,Marisco,https://www.google.com/maps/place/Restaurante+...,4.603966,-74.074317,93cb8caef719bf99a37bb12b503ee75a
4,Restaurante Cafetería Aqúm,NaN,NaN,None,Restaurante,https://www.google.com/maps/place/Restaurante+...,4.603760,-74.075963,9b2d5b4678781e53038e91ea5324530a
...,...,...,...,...,...,...,...,...,...
225,Anna & Otto Macarena,4.6,672.0,Cra. 4a #26c-35,Pizza,https://www.google.com/maps/place/Anna+%26+Ott...,4.613256,-74.066599,f015860f80a926525974517549239c6b
226,Pastis Ristorante,4.4,46.0,Cra. 4a #27 - 86,Italiana,https://www.google.com/maps/place/Pastis+Risto...,4.614742,-74.066085,41462500978ad1b6864f594a603b3061
227,Restaurante rincón Casa de Paella,4.5,172.0,Cl. 26c #4a-15,Restaurante,https://www.google.com/maps/place/Restaurante+...,4.612743,-74.066768,e01677ef20fc43fcf6dea862c590e0b9
228,Bangkok Thai Cuisine,4.5,192.0,Cl. 26c #4-41,Tailandesa,https://www.google.com/maps/place/Bangkok+Thai...,4.612694,-74.066522,b45cbcc226145cbdceae9bb2a8a87c33


In [173]:
def find_polygon_containing_point(geojson, lng, lat):
    point_feature = Point([lng, lat])
    for feature in geojson['features']:
        try:
            if boolean_point_in_polygon(point_feature, feature):
                return feature['properties']['nom_upz']
        except Exception as e:
            print(e)
    print("Upz not available")
    return None  # No polygon found

In [172]:
df['upz'] = df.apply(lambda row: find_polygon_containing_point(geojson,row['longitud'], row['latitud']), axis=1)
df['upz'].unique()

{"coordinates": [-74.076967, 4.603997], "type": "Point"}
{"coordinates": [-74.07678, 4.605092], "type": "Point"}
{"coordinates": [-74.076309, 4.603622], "type": "Point"}
{"coordinates": [-74.074317, 4.603966], "type": "Point"}
{"coordinates": [-74.075963, 4.60376], "type": "Point"}
{"coordinates": [-74.074853, 4.602729], "type": "Point"}
{"coordinates": [-74.07438, 4.603489], "type": "Point"}
{"coordinates": [-74.074129, 4.603033], "type": "Point"}
{"coordinates": [-74.07606, 4.602782], "type": "Point"}
{"coordinates": [-74.076861, 4.603809], "type": "Point"}
{"coordinates": [-74.074393, 4.603862], "type": "Point"}
{"coordinates": [-74.07491, 4.603107], "type": "Point"}
{"coordinates": [-74.076345, 4.605034], "type": "Point"}
{"coordinates": [-74.074308, 4.604473], "type": "Point"}
{"coordinates": [-74.076318, 4.603568], "type": "Point"}
{"coordinates": [-74.074161, 4.60483], "type": "Point"}
{"coordinates": [-74.073972, 4.604519], "type": "Point"}
{"coordinates": [-74.075948, 4.603751

{"coordinates": [-74.071082, 4.610458], "type": "Point"}
{"coordinates": [-74.071667, 4.609767], "type": "Point"}
{"coordinates": [-74.071599, 4.609396], "type": "Point"}
{"coordinates": [-74.070975, 4.609215], "type": "Point"}
{"coordinates": [-74.07083, 4.60924], "type": "Point"}
{"coordinates": [-74.070965, 4.610121], "type": "Point"}
{"coordinates": [-74.070606, 4.609225], "type": "Point"}
{"coordinates": [-74.070491, 4.610254], "type": "Point"}
{"coordinates": [-74.070434, 4.609425], "type": "Point"}
{"coordinates": [-74.07027, 4.609239], "type": "Point"}
{"coordinates": [-74.069995, 4.608692], "type": "Point"}
{"coordinates": [-74.070243, 4.609464], "type": "Point"}
{"coordinates": [-74.07043, 4.609438], "type": "Point"}
{"coordinates": [-74.070183, 4.611085], "type": "Point"}
{"coordinates": [-74.070636, 4.609139], "type": "Point"}
{"coordinates": [-74.070364, 4.6102], "type": "Point"}
{"coordinates": [-74.069617, 4.608431], "type": "Point"}
{"coordinates": [-74.065754, 4.611758

array(['LAS NIEVES', 'LA MACARENA', 'LA CANDELARIA', 'LA SABANA',
       'SAGRADO CORAZON', 'TEUSAQUILLO'], dtype=object)

In [176]:
df

,name,rating,number_commnets_google,address,type,google_maps_url,latitud,longitud,hash_id,upz
0,Restaurante Morena Parrilla,4.0,17.0,Cra. 12 # 15-25 Piso 2. Centro Comercial Gran...,Colombiana,https://www.google.com/maps/place/Restaurante+...,4.603997,-74.076967,027bcb6af9d1be568cf1ae518fa7c10f,LAS NIEVES
1,RESTAURANTE GUSTA MAS,5.0,2.0,Cl. 16 #12-55,Colombiana,https://www.google.com/maps/place/RESTAURANTE+...,4.605092,-74.076780,c52a652d8a563dc0cd251ddf5814bbe2,LAS NIEVES
2,Restaurante El Antojo de La 15,NaN,NaN,Cl. 15 #32,Restaurante,https://www.google.com/maps/place/Restaurante+...,4.603622,-74.076309,b6bc338fc1a8d79b22083f3d315dc6be,LAS NIEVES
3,Restaurante Pescadero Del Centro,4.5,40.0,Cra. 9 #58,Marisco,https://www.google.com/maps/place/Restaurante+...,4.603966,-74.074317,93cb8caef719bf99a37bb12b503ee75a,LAS NIEVES
4,Restaurante Cafetería Aqúm,NaN,NaN,None,Restaurante,https://www.google.com/maps/place/Restaurante+...,4.603760,-74.075963,9b2d5b4678781e53038e91ea5324530a,LAS NIEVES
...,...,...,...,...,...,...,...,...,...,...
225,Anna & Otto Macarena,4.6,672.0,Cra. 4a #26c-35,Pizza,https://www.google.com/maps/place/Anna+%26+Ott...,4.613256,-74.066599,f015860f80a926525974517549239c6b,LA MACARENA
226,Pastis Ristorante,4.4,46.0,Cra. 4a #27 - 86,Italiana,https://www.google.com/maps/place/Pastis+Risto...,4.614742,-74.066085,41462500978ad1b6864f594a603b3061,LA MACARENA
227,Restaurante rincón Casa de Paella,4.5,172.0,Cl. 26c #4a-15,Restaurante,https://www.google.com/maps/place/Restaurante+...,4.612743,-74.066768,e01677ef20fc43fcf6dea862c590e0b9,LA MACARENA
228,Bangkok Thai Cuisine,4.5,192.0,Cl. 26c #4-41,Tailandesa,https://www.google.com/maps/place/Bangkok+Thai...,4.612694,-74.066522,b45cbcc226145cbdceae9bb2a8a87c33,LA MACARENA


In [177]:
df.to_json(path_or_buf='D:/MarketResearch/Restaurantes/restaurants_sample.json', orient='records')